<a href="https://colab.research.google.com/github/PatrikDurdevic/Don-t-Patronize-Me-FER/blob/main/SVM_WV_Task1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from urllib import request
module_url = f"https://raw.githubusercontent.com/Perez-AlmendrosC/dontpatronizeme/master/semeval-2022/dont_patronize_me.py"
module_name = module_url.split('/')[-1]
print(f'Fetching {module_url}')
#with open("file_1.txt") as f1, open("file_2.txt") as f2
with request.urlopen(module_url) as f, open(module_name,'w') as outf:
  a = f.read()
  outf.write(a.decode('utf-8'))

Fetching https://raw.githubusercontent.com/Perez-AlmendrosC/dontpatronizeme/master/semeval-2022/dont_patronize_me.py


In [1]:
import pandas as pd
import numpy as np
import spacy
from dont_patronize_me import DontPatronizeMe

In [2]:
dpm = DontPatronizeMe('.', 'dontpatronizeme_pcl.tsv')
dpm.load_task1()

In [13]:
EMBEDDING_FILE = '/root/input/GoogleNews-vectors-negative300.bin.gz'
!wget -P /root/input/ -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"
!python -m spacy download en_core_web_lg

--2021-11-07 14:31:00--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.196.48
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.196.48|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1647046227 (1.5G) [application/x-gzip]
Saving to: ‘/root/input/GoogleNews-vectors-negative300.bin.gz’

GoogleNews-vectors- 100%[===================>]   1.53G  41.0MB/s    in 37s     

2021-11-07 14:31:37 (42.5 MB/s) - ‘/root/input/GoogleNews-vectors-negative300.bin.gz’ saved [1647046227/1647046227]



In [3]:
from gensim.models import KeyedVectors
model = KeyedVectors.load_word2vec_format('../root/input/GoogleNews-vectors-negative300.bin.gz', binary=True)

In [11]:
import nltk
from nltk.corpus import stopwords

In [25]:
def preprocess(sentence):
  nltk.word_tokenize(sentence)
  sentence = [word for word in sentence if word not in stopwords.words('english')]
  sentence = [model.get_vector(w) for w in sentence if w in model.vocab]
  return np.stack(sentence, axis=0).mean(0).reshape(300,) if len(sentence) > 0 else np.zeros(300,)

In [28]:
from tqdm import tqdm
tqdm.pandas()
dpm.train_task1_df["text_vectorized_2"] = dpm.train_task1_df["text"].progress_apply(lambda x: preprocess(x))

100%|██████████| 10469/10469 [06:22<00:00, 27.36it/s]


In [29]:
from sklearn import svm
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import classification_report
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

SVM = svm.SVC(C=10.0, kernel='poly', gamma='scale')
for train_index, test_index in sss.split(dpm.train_task1_df["text_vectorized_2"], dpm.train_task1_df["label"]):
  X_train, X_test = np.stack(dpm.train_task1_df["text_vectorized_2"].iloc[train_index], axis=0), np.stack(dpm.train_task1_df["text_vectorized_2"].iloc[test_index], axis=0)
  y_train, y_test = dpm.train_task1_df["label"].iloc[train_index], dpm.train_task1_df["label"].iloc[test_index]

  SVM.fit(X_train, y_train)

  predictions_SVM = SVM.predict(X_test)
  print(classification_report(y_test, predictions_SVM))

              precision    recall  f1-score   support

           0       0.90      1.00      0.95      1895
           1       0.00      0.00      0.00       199

    accuracy                           0.90      2094
   macro avg       0.45      0.50      0.48      2094
weighted avg       0.82      0.90      0.86      2094



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [40]:
sentence = input()

we 're living in times of absolute insanity , as i 'm pretty sure most people are aware . for a while , waking up every day to check the news seemed to carry with it the same feeling of panic and dread that action heroes probably face when they 're trying to decide whether to cut the blue or green wire on a ticking bomb -- except the bomb 's instructions long ago burned in a fire and imminent catastrophe seems the likeliest outcome . it 's hard to stay that on-edge for that long , though , so it 's natural for people to become inured to this constant chaos , to slump into a malaise of hopelessness and pessimism .


In [47]:
sentence_preprocessed = preprocess(sentence)
print(SVM.predict(sentence_preprocessed.reshape(1, -1))[0])

0
